In [4]:
!pip install gym
!pip install torch
!pip install torchvision

In [33]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image
import copy

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

In [7]:
torch.cuda.is_available()

False

In [34]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [35]:
device

device(type='cpu')

In [10]:
agent = nn.Sequential(
    nn.Linear(2, 32),
    nn.ReLU(),
    nn.Linear(32, 32),
    nn.ReLU(),
    nn.Linear(32, 3)
)

In [11]:
Transition = namedtuple('Transition', 'state action reward next_state done')

In [15]:
class Cache:
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = [None for _ in range(capacity)]
        self.index = 0
        
    
    def push(self, x):
        self.memory[self.index] = x
        self.index = (self.index + 1) % self.capacity
        
    
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    
    def full(self):
        return len(self.memory) == self.capacity

In [43]:
class Agent:
    def __init__(self, input_dim, hidden_layers_dim, output_dim, lr, optimizer, loss_function
                 , gamma, eps, eps_decay, eps_min_bound, cache_size, batch_size):
        
        self.policy_net = nn.Sequential(
            nn.Linear(input_dim, hidden_layers_dim),
            nn.ReLU(),
            nn.Linear(hidden_layers_dim, hidden_layers_dim),
            nn.ReLU(),
            nn.Linear(hidden_layers_dim, output_dim)
        )
        self.target_net = copy.deepcopy(self.policy_net)
        
        self.policy_net.to(device)
        self.target_net.to(device)
        
        self.optimizer = optimizer(self.policy_net.parameters(), lr=lr)
        self.loss_function = loss_function
        
        self.input_dim = input_dim
        self.hidden_layers_dim = hidden_layers_dim
        self.output_dim = output_dim
        
        self.gamma = gamma
        self.eps = eps
        self.eps_decay = eps_decay
        self.eps_min_bound = eps_min_bound
        
        self.cache = Cache(cache_size)
        self.batch_size = batch_size
        
    
    def select_action(self, state):
        self.eps *= self.eps_decay
        self.eps = max(self.eps, self.eps_min_bound)
        
        if random.random() < self.eps:
            return random.randint(0, self.output_dim)
        return self.policy_net().argmax().item()
    
    
    def fit(self):
        batch = self.buffer.sample(self.batch_size)
        
        states = torch.stack([b.state for b in batch]).to(device)
        next_states = torch.stack([b.next_state for b in batch]).to(device)
        
        actions = torch.tensor([b.action for b in batch]).to(device)
        dones = torch.tensor([b.done] for b in batch).to(device)
        
        rewards = torch.tensor([b.rewards for b in batch]).to(device).float()
        
        target_q = torch.zeros(len(batch_size)).to(device).float()
        with torch.no_grad():
            target_q = target_net(next_states).max(1)[0]
        target_q[done] = 0
        target_q = rewards + target_q * self.gamma
        
        actual_q = policy_net(states).gather(1, actions)
        
        print(target_q, actual_q)
        exit()
        
        loss = self.loss_function()

In [39]:
env = gym.make('MountainCar-v0')

In [44]:
agent = Agent(2, 16, 3, 0.0003, optim.Adam, F.smooth_l1_loss, 0.99, 0.9, 0.999, 0.1, 5000, 100)

In [37]:
n_games = 20
for i in range(n_games):
    state = env.reset()
    state
    done = False
    while not done:
        action = select_action(state)
        next_state, reward, done, _ = env.step(action)
        agent.cache.store_transition()

Transition(state=1, action=2, reward=3, next_state=4, done=5)